In [1]:
import os
os.chdir('..')

In [10]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import tqdm

from src.data.arma import (load_video_frames,
                           convert_annotation_boar_points_to_np_arrays, 
                           prepare_referential_timeseries_df,
                           merge_annotation_with_referential_time_df,
                           interpolate_boar_positions,
                           find_bboxes_for_frame,
                           find_bboxes_for_video,
                           prepare_frames_with_drawings,
                           save_video_mp4)

In [14]:
annot = pd.read_csv('data/arma_generated/20fps_sample/7.csv', parse_dates=['timestamp'])
annot = convert_annotation_boar_points_to_np_arrays(annot)
annot.head()

,timestamp,frame,objects
0,2018-12-10 16:22:04,1,"[[454.903, -21.7134], [476.824, -9.25929], [49..."
1,2018-12-10 16:22:04,2,"[[454.901, -17.7646], [476.782, -5.20606], [49..."
2,2018-12-10 16:22:04,3,"[[454.902, -19.3441], [476.782, -6.78994], [49..."
3,2018-12-10 16:22:04,12,"[[454.903, -22.5032], [476.828, -10.2306], [49..."
4,2018-12-10 16:22:04,14,"[[454.902, -20.9236], [476.824, -8.46759], [49..."


In [ ]:
frames = load_video_frames('data/arma_generated/20fps_sample/2018-12-10_16-22-04.mp4')
FRAME_RATE = 20
len(frames)

In [ ]:
start_time = datetime.strptime('2018-12-10 16:22:04', '%Y-%m-%d %H:%M:%S')
duration = timedelta(seconds=int(len(frames)/FRAME_RATE))

referential_time_df = prepare_referential_timeseries_df(annot, start_time, duration, FRAME_RATE)
referential_time_df.head()

In [7]:
annotation = merge_annotation_with_referential_time_df(annot, referential_time_df)
annotation.head()

,timestamp,frame,objects
0,2018-12-10 16:22:04,0,NaN
1,2018-12-10 16:22:04,1,"[[454.903, -21.7134], [476.824, -9.25929], [49..."
2,2018-12-10 16:22:04,2,"[[454.901, -17.7646], [476.782, -5.20606], [49..."
3,2018-12-10 16:22:04,3,"[[454.902, -19.3441], [476.782, -6.78994], [49..."
4,2018-12-10 16:22:04,4,NaN


In [8]:
annotation['objects'] = interpolate_boar_positions(annotation['objects'])
annotation.head()

,timestamp,frame,objects
0,2018-12-10 16:22:04,0,"[[454.903, -21.7134], [476.824, -9.25929], [49..."
1,2018-12-10 16:22:04,1,"[[454.903, -21.7134], [476.824, -9.25929], [49..."
2,2018-12-10 16:22:04,2,"[[454.901, -17.7646], [476.782, -5.20606], [49..."
3,2018-12-10 16:22:04,3,"[[454.902, -19.3441], [476.782, -6.78994], [49..."
4,2018-12-10 16:22:04,4,"[[454.902, -19.3441], [476.782, -6.78994], [49..."


In [11]:
annotation = find_bboxes_for_video(frames, annotation, progressbar=tqdm.tqdm_notebook)

In [12]:
annotated_frames = prepare_frames_with_drawings(frames, annotation, progressbar=tqdm.tqdm_notebook)

In [13]:
save_video_mp4(annotated_frames, 'temp.mp4')